In [14]:
from math import log2
import pandas as pd
from ID3 import ID3


In [15]:
dx = {"Outlook"    : ["sunny", "sunny", "overcast", "rain", "rain", 
                      "rain", "overcast", "sunny", "sunny", "rain", 
                      "sunny", "overcast", "overcast", "rain"],

     "Temperature" : ["hot", "hot", "hot", "mild", "cool", 
                      "cool", "cool", "mild", "cool", "mild", 
                      "mild", "mild", "hot", "mild"],

     "Humidity"    : ["high", "high", "high", "high", "normal",
                      "normal", "normal", "high", "normal", "normal",
                      "normal", "high", "normal", "high"],

     "Windy"       : ["false", "true", "false", "false", "false",
                      "true", "true", "false", "false", "false",
                      "true", "true", "false", "true"]}

dy = ["N", "N", "P", "P", "P", 
      "N", "P", "N", "P", "P", 
      "P", "P", "P", "N"]

X = pd.DataFrame(dx)
Y = pd.DataFrame(dy)
X.index = X.index + 1
Y.index = Y.index + 1


In [16]:
from sklearn.preprocessing import LabelEncoder

X = X.apply(LabelEncoder().fit_transform)
Y = Y.apply(LabelEncoder().fit_transform)


In [17]:
dt = ID3()
dt.fit(X,Y)


In [18]:
vars(dt.root)

print(dt.root.split_attribute)

def H(ps):
    return -sum(p*log2(p) for p in ps)

# windy.
ps = [6/14, 8/14]
print(H(ps))

# humidity.
ps = [7/14, 7/14]
print(H(ps))

# temperature.
ps = [4/14, 6/14, 4/14]
print(H(ps))

# outlook.
ps = [5/14, 4/14, 5/14]
print(H(ps))

root = dt.root
root.children

overcast_child = dt.root.children[0]
rain_child = dt.root.children[1]
sunny_child = dt.root.children[2]
print(overcast_child.class_proba)


Outlook
0.9852281360342515
1.0
1.5566567074628228
1.5774062828523454
[0.0, 1.0]


In [19]:
sunny_child.children
sunny_high = sunny_child.children[0]
sunny_normal = sunny_child.children[1]
print(sunny_child.split_attribute)


Humidity


In [20]:
print(sunny_high.class_proba)
print(sunny_normal.class_proba)


[1.0, 0.0]
[0.0, 1.0]


In [21]:
rain_child.children
rain_false = rain_child.children[0]
rain_true = rain_child.children[1]
print(rain_child.split_attribute)


Windy


In [22]:
print(rain_false.class_proba)
print(rain_true.class_proba)
print(root.class_proba)
print(sunny_child.class_proba)


[0.0, 1.0]
[1.0, 0.0]
[0.35714285714285715, 0.6428571428571429]
[0.6, 0.4]


In [25]:
dx_test = {"Outlook"    : ["sunny", "sunny", "overcast", "rain", "overcast"],
          "Temperature" : ["hot", "hot", "cool", "mild", "mild"],
          "Humidity"    : ["high", "high", "high", "normal", "normal"],
          "Windy"       : ["false", "true", "false", "true", "false"]}

X_test = pd.DataFrame(dx_test)
X_test = X_test.apply(LabelEncoder().fit_transform)
X_test


,Outlook,Temperature,Humidity,Windy
0,2,1,0,0
1,2,1,0,1
2,0,0,0,0
3,1,2,1,1
4,0,2,1,0


In [27]:
row_data = X_test.iloc[3]
ID3._predict_proba_row(row_data, dt.root)
probs = dt.predict_proba(X_test)
print(probs)
print(dt.predict(X_test)) # 0 0 1 0 1
# One sticky point -- encoding for testing must agree with the encoding for training.  


     0    1
0  1.0  0.0
1  1.0  0.0
2  0.0  1.0
3  1.0  0.0
4  0.0  1.0
0    0
1    0
2    1
3    0
4    1
dtype: int64
